In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir("/content/drive/MyDrive/ml4nlp2/final_folder")
!ls

from  IPython. display  import  clear_output
import warnings
warnings. filterwarnings('ignore')


# installs
!pip install datasets
!pip install transformers==4.28.0
!pip install sentencepiece==0.1.98
!pip install pysentimiento

#imports
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path
import pandas as pd
import numpy as np
import random
import  matplotlib. pyplot  as  plt
from tqdm import tqdm

import torch
import torch.optim as optim
import  torch. nn. functional  as  F

#set device
import torch
# device = "cpu"
device  =  torch. device('cuda:0'  if  torch. cuda. is_available() else  'cpu')
print(f"computation will run on {device} now")

Mounted at /content/drive
 baselines			     logs
 converted_og			    'models_zoo -rohit.xlsx'
 data_process_code		     models_zoo.xlsx
 dev_data_task1.csv		     og_data
 dev_data_task1_en.csv		     output
 dev_data_task1_es.csv		     output_hard1.json
 dev_data_task2.csv		     output_soft1.json
 dev_data_task2_en.csv		     rohit_final_code
 dev_data_task2_es.csv		     test_final.csv
 dev_data_task3.csv		     train_data_task1.csv
 dev_data_task3_en.csv		     train_data_task1_en.csv
 dev_data_task3_es.csv		     train_data_task1_es.csv
 exist2023evaluation.py		     train_data_task2.csv
 EXIST2023_test_clean.json	     train_data_task2_en.csv
 golds				     train_data_task2_es.csv
 good_paper_distilled_learning.pdf   train_data_task3.csv
 hp_optune			     train_data_task3_en.csv
 large_train_data_task1.csv	     train_data_task3_es.csv
 large_train_data_task1_en.csv	     translated_og
 large_train_data_task1_es.csv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

## hard

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

#instantiate label encoders
from sklearn.preprocessing import LabelEncoder
task_encoder = LabelEncoder()

def task_hard_encode(df):
    task_encoder.fit(all_task_hard_labels)
    df['hard_label'] = task_encoder.transform(df['hard_label'])
    return df

def task_hard_decode(df):
    df["hard_label"] = task_encoder.inverse_transform(df["hard_label"])
    return df


from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset


#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task2.csv")
og_dev1 = pd.read_csv("dev_data_task2.csv")
og_test1 = pd.read_csv("test_final.csv")

all_task_hard_labels = pd.concat([og_train1["hard_label"],og_dev1["hard_label"]])
train1_df = task_hard_encode(og_train1)

# print(train1_df.columns)
train1_df = train1_df[["tweet","hard_label"]].dropna()
train1_df = train1_df[train1_df['hard_label'] != 4]
train1_df["tweet"] = simple_preprocess(train1_df["tweet"])

dev1_df = task_hard_encode(og_dev1)
dev1_df = dev1_df[["tweet","hard_label"]].dropna()
dev1_df = dev1_df[dev1_df['hard_label'] != 4]
dev1_df["tweet"] = simple_preprocess(dev1_df["tweet"])

test1_df = og_test1
test1_df = test1_df[["tweet"]]
# test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)

import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True)

# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    # accelerator = Accelerator(),
    output_dir="./output",
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=2e-5, #replaced with optuna best, original is 2e-05, 0.0004005073373276802
    weight_decay=0.0053619187660176775, #replaced with opunta best, original is 0.01
    logging_dir="./logs",
    # metric_for_best_model="f1_score",
    # greater_is_better=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)

def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["hard_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})


# Define the trainer for each fold
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_labels = np.argmax(val_predictions.predictions, axis=1)
val_true_labels = val_dataset["label"]

val_accuracy = accuracy_score(val_true_labels, val_pred_labels)
val_f1_score = f1_score(val_true_labels, val_pred_labels,average = "weighted")

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

train1 (5496, 2)
<bound method NDFrame.head of                                                   tweet  hard_label
0     @USER Ignora al otro, es un capullo.El problem...           3
1     @USER Si comicsgate se parece en algo a gamerg...           2
2     @USER Lee sobre Gamergate, y como eso ha cambi...           2
5     @USER Aaah sí. Andrew Dobson. El que se dedicó...           2
7     @USER Esta gringa sigue llorando por el gamerg...           0
...                                                 ...         ...
6913  @USER Ma'am if I say that you look like a whor...           0
6915  idk why y’all bitches think having half your a...           0
6916  This has been a part of an experiment with @US...           1
6917  "Take me already" "Not yet. You gotta be ready...           0
6918    @USER why do you look like a whore? /lh HTTPURL           0

[5496 rows x 2 columns]>
test1 (2076, 1)
<bound method NDFrame.head of                                                   tweet
0     @US

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dens

Epoch,Training Loss,Validation Loss
1,0.768200,0.597305


In [ ]:
# Get predictions on the test set
test_predictions = trainer.predict(test_dataset)
test_pred_labels = np.argmax(test_predictions.predictions, axis=1)
test_pred_probabilities = softmax(test_predictions.predictions).tolist()

In [ ]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

test_probs_list = create_results(test_pred_probabilities, task_encoder.classes_)
test_probs_list

In [ ]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id_EXIST"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = ""
    print("output generated as json")
    with open('output_soft3.json', 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels,columns=["hard_label"]), test_probs_list, og_test1[["id_EXIST"]],og_test1)

In [ ]:
outputs_json

## soft

In [ ]:
#relevant functions
import re
# Create function for printing 
def print_custom(text):
    print('\n')
    print(text)
    print('-'*100)
    
def simple_preprocess(text):
  """
  pass the tweet data as a series. do not use apply function
  only preprocesses for replacing @USER and URLS
  """
  # print("i am preprocessing")
  URL_RE = re.compile(r"https?:\/\/[\w\.\/\?\=\d&#%_:/-]+")
  HANDLE_RE = re.compile(r"@\w+")
  tweets = []
  for t in text:
    t = HANDLE_RE.sub("@USER", t)
    t = URL_RE.sub("HTTPURL", t)
    tweets.append(t)
  return tweets

def convert_logits_to_list(logits_dict):
    """
    order:
    1. classes are 
    REPORTED, NO, DIRECT, JUDGEMENTAL 
    """
    logits_dict = eval(logits_dict)
    logits_list = [logits_dict["REPORTED"], logits_dict["NO"],logits_dict["DIRECT"],logits_dict["JUDGEMENTAL"]]
    return logits_list

def convert_list_to_logits(logits_list):
    logits_dict = {"REPORTED": logits_list[0], "NO": logits_list[1],"DIRECT": logits_list[2], "JUDGEMENTAL": logits_list[3]}
    return logits_dict

def check_dtype(given_data):
    return eval(given_data)

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer  
import os.path
from os import path

from transformers import XLMRobertaForSequenceClassification, XLMRobertaTokenizer, TrainingArguments, Trainer
from datasets import Dataset


#load given data
import pandas as pd
og_train1 = pd.read_csv("train_data_task2.csv")
og_dev1 = pd.read_csv("dev_data_task2.csv")
og_test1 = pd.read_csv("test_final.csv")

og_train1["soft_label"] = og_train1["soft_label"].apply(convert_logits_to_list)
og_train1["tweet"] = simple_preprocess(og_train1["tweet"])
# og_train1["soft_label"] = og_train1["soft_label"].apply(check_dtype) 
train1_df = og_train1[["tweet","soft_label"]].dropna()

og_dev1["soft_label"] = og_dev1["soft_label"].apply(convert_logits_to_list)
og_dev1["tweet"] = simple_preprocess(og_dev1["tweet"])
# og_dev1["soft_label"] = og_dev1["soft_label"].apply(check_dtype) 
dev1_df = og_dev1[["tweet","soft_label"]].dropna()


test1_df = og_test1
test1_df = test1_df[["tweet"]]
# test1_df = test1_df[test1_df['hard_label'] != 2]
test1_df["tweet"] = simple_preprocess(test1_df["tweet"])

print("train1",train1_df.shape)
print(train1_df.head)
print("test1",test1_df.shape)
print(test1_df.head)


import pandas as pd
from sklearn.model_selection import train_test_split

# Combine train1df and test1df into a single dataframe
combined_df = pd.concat([train1_df, dev1_df], ignore_index=True)

# Shuffle the combined dataframe
combined_df_shuffled = combined_df.sample(frac=1, random_state=42)

# Split the shuffled dataframe into train, validation, and test dataframes with an 80-10-10 split
train_df, val_df = train_test_split(combined_df_shuffled, test_size=0.15, random_state=42)
test_df = test1_df

# Reset the indices of the dataframes
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


# Load the tokenizer and model
model_name = "sdadas/xlm-roberta-large-twitter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4,ignore_mismatched_sizes=True)

# Set parameters
MAX_LENGTH = 128

# Define the training arguments
training_args = TrainingArguments(
    # accelerator = Accelerator(),
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    learning_rate=2e-5, #replaced with optuna best, original is 2e-05, 0.0004005073373276802
    weight_decay=0.0053619187660176775, #replaced with opunta best, original is 0.01
    logging_dir="./logs",
    # metric_for_best_model="f1_score",
    # greater_is_better=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch"
)

import torch.nn as nn
def convert_to_dataset(df):
    df = {"text": df['tweet'].tolist(), "label": df["soft_label"].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset

def convert_to_dataset_test(df):
    df = {"text": df['tweet'].tolist()}
    dataset = Dataset.from_dict(df)
    return dataset


# Convert dataframe to dataset
train_dataset = convert_to_dataset(train_df)
val_dataset = convert_to_dataset(val_df)
test_dataset = convert_to_dataset_test(test_df)


# Create the datasets
train_encodings = tokenizer(train_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
train_dataset = Dataset.from_dict({"input_ids": train_encodings["input_ids"], "attention_mask": train_encodings["attention_mask"], "label": train_dataset["label"]})

val_encodings = tokenizer(val_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
val_dataset = Dataset.from_dict({"input_ids": val_encodings["input_ids"], "attention_mask": val_encodings["attention_mask"], "label": val_dataset["label"]})


test_encodings = tokenizer(test_dataset["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH)
test_dataset = Dataset.from_dict({"input_ids": test_encodings["input_ids"], "attention_mask": test_encodings["attention_mask"]})

def custom_loss_fn(logits, soft_labels):
    probs = F.softmax(logits, dim=1)
    # Apply nn.CrossEntropyLoss
    loss = nn.CrossEntropyLoss(reduction="sum",label_smoothing=0.15)(probs, soft_labels)
    return loss


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

# Define the trainer for each fold
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train the model for each fold
trainer.train()

from scipy.special import softmax
from sklearn.metrics import accuracy_score, f1_score
# Get predictions on the validation set
val_predictions = trainer.predict(val_dataset)
val_pred_logit_labels = val_predictions.predictions
val_pred_probabilities = softmax(val_pred_logit_labels, axis=1)

val_pred_hard_labels = np.argmax(val_pred_probabilities, axis=1)
val_true_labels = val_dataset["label"]
val_true_hard_labels = np.argmax(val_dataset["label"],axis=1)

# Calculate evaluation metrics for each fold
val_accuracy = accuracy_score(val_true_hard_labels, val_pred_hard_labels)
val_f1_score = f1_score(val_true_hard_labels, val_pred_hard_labels,average='weighted')

print("Validation Accuracy:", val_accuracy)
print("Validation F1 Score:", val_f1_score)

train1 (6920, 2)
<bound method NDFrame.head of                                                   tweet  \
0     @USER Ignora al otro, es un capullo.El problem...   
1     @USER Si comicsgate se parece en algo a gamerg...   
2     @USER Lee sobre Gamergate, y como eso ha cambi...   
3     @USER Un retraso social bastante lamentable, g...   
4     @USER @USER @USER Entonces como así es el merc...   
...                                                 ...   
6915  idk why y’all bitches think having half your a...   
6916  This has been a part of an experiment with @US...   
6917  "Take me already" "Not yet. You gotta be ready...   
6918    @USER why do you look like a whore? /lh HTTPURL   
6919  ik when mandy says “you look like a whore” i l...   

                                             soft_label  
0     [0.5, 0.16666666666666602, 0.0, 0.333333333333...  
1     [0.0, 0.833333333333333, 0.16666666666666602, ...  
2                                  [0.0, 1.0, 0.0, 0.0]  
3     [0.333

Some weights of the model checkpoint at sdadas/xlm-roberta-large-twitter were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at sdadas/xlm-roberta-large-twitter and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dens

Epoch,Training Loss,Validation Loss
1,9.562500,18.596527


In [ ]:
# Get predictions on the test set
test_predictions1 = trainer.predict(test_dataset)
test_pred_labels1 = np.argmax(test_predictions1.predictions, axis=1)
test_pred_probabilities1 = softmax(test_predictions1.predictions).tolist()

In [ ]:
def create_results(probabilities, class_labels):
    results = []
    for prob_list in probabilities:
        result_dict = {label: prob for label, prob in zip(class_labels, prob_list)}
        results.append(result_dict)
    return results

classes_labels = ["REPORTED", "NO", "DIRECT", "JUDGEMENTAL"]

test_probs_list1 = create_results(test_pred_probabilities1, classes_labels)
test_probs_list1

In [ ]:
import json
def get_json_for_evaluation(df, probs_list, ids, gold):
    gold["index"] = gold["id_EXIST"].astype(int)
    output_dict = {}
    decoded_labels = task_hard_decode(df)["hard_label"].tolist()
    print("decoded labels length:",len(decoded_labels))
    for i, row in enumerate(probs_list):
        soft_label = row
        hard_label = decoded_labels[i]
        local_dict = {"hard_label":hard_label,"soft_label": soft_label}
        output_dict[int(gold["index"][i])] = local_dict
    
    filename = "/content/drive/MyDrive/ml4nlp2/for_submission/group-submissions/exist2023_roh-neil/task2_roh-neil_3.json"
    print("output generated as json")
    with open(filename, 'w') as file:
        json.dump(output_dict, file, indent=4)
    return output_dict

outputs_json = get_json_for_evaluation(pd.DataFrame(test_pred_labels1,columns=["hard_label"]), test_probs_list1, og_test1[["id_EXIST"]],og_test1)